In [1]:
import os, argparse

import tensorflow as tf

from tensorflow.python.framework import graph_util



dir = os.path.dirname("C:/Users/Thiya/Desktop/Wayne/Sofwerx/Project/photos1/face2face_reduced")





def freeze_graph(model_folder):

    # We retrieve our checkpoint fullpath
    import pdb
    

    checkpoint = tf.train.get_checkpoint_state(model_folder)
    

    input_checkpoint = checkpoint.model_checkpoint_path

    input_checkpoint = input_checkpoint.replace('//', '/')

    # We precise the file fullname of our freezed graph

    absolute_model_folder = '/'.join(input_checkpoint.split('/')[:-1])
    

    output_graph = absolute_model_folder + '/frozen_model.pb'



    # Before exporting our graph, we need to precise what is our output node

    # This is how TF decides what part of the Graph he has to keep and what part it can dump

    # NOTE: this variable is plural, because you can have multiple output nodes

    output_node_names ='generate_output/output'

    print(output_node_names)

    # We clear devices to allow TensorFlow to control on which device it will load operations

    clear_devices = True



    # We import the meta graph and retrieve a Saver

    saver = tf.train.import_meta_graph(input_checkpoint + '.meta', clear_devices=clear_devices)


    
    # We retrieve the protobuf graph definition

    graph = tf.get_default_graph()

    input_graph_def = graph.as_graph_def()
    
    #pdb.set_trace()

    # We start a session and restore the graph weights

    init_g = tf.global_variables_initializer()
    init_l = tf.local_variables_initializer()

    
    with tf.Session() as sess:
        sess.run(init_g)
        sess.run(init_l)
        saver.restore(sess,input_checkpoint)

  

        # We use a built-in TF helper to export variables to constants
        
        output_graph_def = graph_util.convert_variables_to_constants(

            sess,  # The session is used to retrieve the weights

            input_graph_def,  # The graph_def is used to retrieve the nodes

            [output_node_names] #.split(",")  # The output node names are used to select the usefull nodes

        )

       

        # Finally we serialize and dump the output graph to the filesystem

        with tf.gfile.GFile(output_graph, 'wb') as f:

            f.write(output_graph_def.SerializeToString())

        print('%d ops in the final graph.' % len(output_graph_def.node))





if __name__ == '__main__':

    parser = argparse.ArgumentParser()

    parser.add_argument('--model-folder', type=str, help='Model folder to export')

    args = parser.parse_known_args()

    
    freeze_graph("C:/Users/Thiya/Desktop/Wayne/Sofwerx/Project/photos1/face2face_reduced/")

C:\Users\Thiya\Anaconda2\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


generate_output/output
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
INFO:tensorflow:Restoring parameters from C:/Users/Thiya/Desktop/Wayne/Sofwerx/Project/photos1/face2face_reduced/reduced_model
INFO:tensorflow:Froze 44 variables.
INFO:tensorflow:Converted 44 variables to const ops.
511 ops in the final graph.
